In [ ]:
import pandas as pd
import numpy as np
import requests
import re
import csv

In [ ]:
pythia_fragment_CP5 = """
from Configuration.Generator.Pythia8CommonSettings_cfi import *
from Configuration.Generator.MCTunesRun3ECM13p6TeV.PythiaCP5Settings_cfi import *
from Configuration.Generator.PSweightsPythia.PythiaPSweightsSettings_cfi import *

generator = cms.EDFilter("Pythia8ConcurrentHadronizerFilter",
                         maxEventsToPrint = cms.untracked.int32(1),
                         pythiaPylistVerbosity = cms.untracked.int32(1),
                         filterEfficiency = cms.untracked.double(1.0),
                         pythiaHepMCVerbosity = cms.untracked.bool(False),
                         comEnergy = cms.double(13600.),
                         PythiaParameters = cms.PSet(
                                                     pythia8CommonSettingsBlock,
                                                     pythia8CP5SettingsBlock,
                                                     pythia8PSweightsSettingsBlock,
                                                     parameterSets = cms.vstring('pythia8CommonSettings',
                                                                                 'pythia8CP5Settings',
                                                                                 'pythia8PSweightsSettings',
                                                                                 )
                                                     )
                         )


ProductionFilterSequence = cms.Sequence(generator)
"""

In [ ]:
def fragmentsDictCreator():
    dict = { '2023': pythia_fragment_CP5,
             '2023BPix': pythia_fragment_CP5
           }
    return dict

In [ ]:
LHEproducer = """
import FWCore.ParameterSet.Config as cms

# link to cards:
# __EXAMPLE__

externalLHEProducer = cms.EDProducer('ExternalLHEProducer',
    args = cms.vstring('__GRIDPACK__'),
    nEvents = cms.untracked.uint32(5000),
    numberOfParameters = cms.uint32(1),
    outputFile = cms.string('cmsgrid_final.lhe'),
    generateConcurrently = cms.untracked.bool(False),
    scriptName = cms.FileInPath('GeneratorInterface/LHEInterface/data/run_generic_tarball_cvmfs.sh')
)
"""

In [ ]:
years = ["2023", "2023BPix"]
M = ["700", "800", "900", "1000", "1100", "1200", "1300", "1400", "1500", "1600", "1700", "1800"]

tot_events = [400000]*len(M)
gridpacks_dict = {}
example_dict = {}
dataset_names = {}

In [ ]:
for year in range(len(years)):
    t_example1 = 'https://github.com/cms-sw/genproductions/tree/master/bin/MadGraph5_aMCatNLO/cards/production/13p6TeV/single_VLQ/NWA/TprimeBToTZ_LH/TprimeBToTZ_LH_narrowLO_M700'
    t_datasetname_year = 'TprimeBtoTZ_M-{massX}_LH_TuneCP5_13p6TeV_madgraph-pythia8'
    t_gp_year = "/cvmfs/cms.cern.ch/phys_generator/gridpacks/RunIII/13p6TeV/slc7_amd64_gcc10/MadGraph5_aMCatNLO/TprimeBToTZ/tZ_b_LH_M{massX}_slc7_amd64_gcc10_CMSSW_12_4_8_tarball.tar.xz"    
    tmp_dataset_dict={}
    tmp_gridpack_dict={}
    tmp_example_dict={}
    dataset_names_year = []
    gp_tprime_year = "gp_tprime_" + years[year]
    gp_tprime_year = []
    
    dataset_names_year = []
    gp_tprime_year = []

    for mx in M:
        d = {"massX": mx}
        tmp_example_dict = t_example1
        gp_tprime_year.append(t_gp_year.format_map(d))
        dataset_names_year.append(t_datasetname_year.format_map(d))
        tmp_dataset_dict = dataset_names_year
        tmp_gridpack_dict = gp_tprime_year
        
    gridpacks_dict[years[year]] = tmp_gridpack_dict
    dataset_names[years[year]] = tmp_dataset_dict
    example_dict[years[year]] = tmp_example_dict

## .csv Production

In [ ]:
#MadgraphVersion = re.compile("V5_2\.[0-9]\.[0-9]")
process_pythia_map = {}
tmp_process_pythia_map = {}
tmp_process_pythia_map = fragmentsDictCreator()
process_pythia_map.update(tmp_process_pythia_map)

In [ ]:
#for year in range(len(years)):
for year in gridpacks_dict:
    #print(year)
    with open('tprimebtotz' + '_' + year +'.csv', 'w') as csvfile:
        csvwriter = csv.writer(csvfile, delimiter=',',
                        quotechar='"', quoting=csv.QUOTE_MINIMAL)
        csvwriter.writerow(['Dataset name','Events', 'fragment','notes','Generator','mcdbid','time','size'])
        #if year == "2022":
        #    pr = 4897
        #if year == "2022EE":
        #    pr = 4748
            
        k=0
        for i in range(len(M)):
            tmp_fragment=""
            #print(gridpacks_dict[year][i])
            #print(dataset_names[year][i])
            #version = MadgraphVersion.search(path).group(0)
            version="2.9.13"
            dataset_name = dataset_names[year][k]
            if year == "2023":
                events = round(tot_events[i]*0.67)
                #prepid = 'B2G-Run3Summer22wmLHEGS-0' + str(pr)
            elif year == "2023BPix":
                events = round(tot_events[i]*0.33)
                #prepid = 'B2G-Run3Summer22wmLHEGS-0' + str(pr)
                
            tmp_fragment = LHEproducer.replace('__GRIDPACK__',gridpacks_dict[year][k]) + '\n' + process_pythia_map[year]
            final_fragment = tmp_fragment.replace('__EXAMPLE__',example_dict[year])
            note = dataset_name.replace('_',' ')
            generators="Madgraph_" + version + "  Pythia8"
            mcdb_id = '0'
            time = '25'
            size = '940'
            csvwriter.writerow([dataset_name, events, final_fragment, note, generators, mcdb_id, time, size])
            #pr = pr+1   
            k=k+1